https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset

In [3]:
import sys
import os
import pandas as pd
import numpy as np
import logging
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from sqlalchemy.exc import SQLAlchemyError


sys.path.append(os.path.abspath('../'))
from src.params import Params
from src.client import DatabaseClient
from src.logging_config import setup_logging

In [4]:
setup_logging()

## Data load

In [5]:
# Parames instance
params = Params()

# Connection to the database
db_client = DatabaseClient(params)

2025-04-04 23:23:13,483 - ERROR - src.client - Failed to connect to the database.
2025-04-04 23:23:13,485 - ERROR - src.client - Error details: (psycopg2.OperationalError) connection to server at "172.23.169.109", port 5433 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)


## Data Overview an Descriptive Statistics

### Overview

The number of observations and features are obtained through Panda's `.shape` method. The "Spotify" dataset contains **114.000 observations (rows)** and **21 features (columns)**.

In [91]:
df.shape

(114000, 21)

The data types are obtained through Panda's `.dtypes` method. The Dataframe contains only 1 boolean feature, 5 object type features, 6 int64 type features and  9 float64 type features.


In [92]:
def columns_per_dtype(df):
    """
    This function returns a dictionary where keys are data types (dtypes)
    and values are lists of column names corresponding to each dtype.

    Parameters:
        df (pd.DataFrame): The DataFrame to analyze.

    Returns:
        dict: A dictionary mapping each dtype to a list of column names with that dtype.
    """
    result = {}
    for col in df.columns:
        col_dtype = df[col].dtype
        if col_dtype not in result:
            result[col_dtype] = []
        result[col_dtype].append(col)
    
    print("Columns grouped by data types:")
    for dtype, columns in result.items():
        print(f"\nData Type: {dtype}")
        print("Columns:")
        for col in columns:
            print(f"  - {col}")


columns_per_dtype(df)

Columns grouped by data types:

Data Type: int64
Columns:
  - Unnamed: 0
  - popularity
  - duration_ms
  - key
  - mode
  - time_signature

Data Type: object
Columns:
  - track_id
  - artists
  - album_name
  - track_name
  - track_genre

Data Type: bool
Columns:
  - explicit

Data Type: float64
Columns:
  - danceability
  - energy
  - loudness
  - speechiness
  - acousticness
  - instrumentalness
  - liveness
  - valence
  - tempo


`Unnamed: 0` column is dropped as it is not part of the original dataset.

In [93]:
df =  df.drop(columns=["Unnamed: 0"])

The duplicated rows are obtained through Panda's `.duplicated` method. The Dataframe has 450 duplicate rows.

In [94]:
df[df.duplicated()].shape[0]

450

The missing values per feature are obtained through Panda's `.isnull().sum()` method. Only the features "artists", "album_name" and "track_name" have missing values, one missing value for each feature. This features, as indicated by their object datatype, are qualitative variables.

In [95]:
df.isnull().sum()

track_id            0
artists             1
album_name          1
track_name          1
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64

Rows with null values are filtered  using the `.isnull()` method combined with `.any(axis=1)`. Only one row contains the missing values.

In [96]:
filtered_rows = df[df.isnull().any(axis=1)]

filtered_rows

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
65900,1kR4gIb7nGxHPI3D2ifs59,NaN,NaN,NaN,0,0,False,0.501,0.583,7,-9.46,0,0.0605,0.69,0.00396,0.0747,0.734,138.391,4,k-pop


The percentage of missing data is aproximmately 0.0001%, which in itself is not very significative.

In [97]:
round(df.isnull().sum().sum() / df.size * 100, 4) 

np.float64(0.0001)

### Descriptive statistics

#### Quantitative variables

Descriptive statistics of quantitative are generated through Panda's `.describe` method.



In [98]:
df.describe()

,popularity,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,114000.000000,1.140000e+05,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000
mean,33.238535,2.280292e+05,0.566800,0.641383,5.309140,-8.258960,0.637553,0.084652,0.314910,0.156050,0.213553,0.474068,122.147837,3.904035
std,22.305078,1.072977e+05,0.173542,0.251529,3.559987,5.029337,0.480709,0.105732,0.332523,0.309555,0.190378,0.259261,29.978197,0.432621
min,0.000000,0.000000e+00,0.000000,0.000000,0.000000,-49.531000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,17.000000,1.740660e+05,0.456000,0.472000,2.000000,-10.013000,0.000000,0.035900,0.016900,0.000000,0.098000,0.260000,99.218750,4.000000
50%,35.000000,2.129060e+05,0.580000,0.685000,5.000000,-7.004000,1.000000,0.048900,0.169000,0.000042,0.132000,0.464000,122.017000,4.000000
75%,50.000000,2.615060e+05,0.695000,0.854000,8.000000,-5.003000,1.000000,0.084500,0.598000,0.049000,0.273000,0.683000,140.071000,4.000000
max,100.000000,5.237295e+06,0.985000,1.000000,11.000000,4.532000,1.000000,0.965000,0.996000,1.000000,1.000000,0.995000,243.372000,5.000000


#### Qualitative variables

Panda's `.describe` method is used with the parameter `include='object'` for describing all qualitative columns of the DataFrame.

In [99]:
df.describe(include='object') 

,track_id,artists,album_name,track_name,track_genre
count,114000,113999,113999,113999,114000
unique,89741,31437,46589,73608,114
top,6S3JlDAGk3uu3NtZbPnuhS,The Beatles,Alternative Christmas 2022,Run Rudolph Run,acoustic
freq,9,279,195,151,1000


1. **Observation Count**: The column `track_id` matches the total number of observations (114,000), which indicates every row has a `track_id` entry.

2. **Uniqueness**: Despite having 114,000 observations, `track_id` contains only 89,741 unique values. This suggests that some `track_id`s are repeated across multiple rows.

4. **track_genre**: This column seems to have very few unique values (only 114), meaning it's highly categorical or repetitive.

5. **Frequent Entries**: The `top` values show the most frequent entry for each column, and `freq` gives the count of that value.  "The Beatles" is the mode in `artists`; "Alternative Christmas 2022"	is the mode in `album_name`; and "Run Rudolph Run" is the mode in `track_name`; and acoustic is the mode in `track_genre`.

## Handling missing values

As stablished previously, only one rows contains missing values, amounting to approximmately 0.0001%, of the data which in itself is not very significative.So this missing data is going to be dropped.

In [100]:
df = df.dropna()

Now the Dataframe has 113.999 rows and 20 columns.

In [101]:
df.shape

(113999, 20)

## Handling duplicated values

As stated previously the Dataframe has 450 duplicate rows.There are going to be dropped.

In [102]:
df = df.drop_duplicates()

In [103]:
print(f"The Dataframe without duplicates has {df.shape[0]} rows and {df.shape[1]} columns")

The Dataframe without duplicates has 113549 rows and 20 columns


### Inspecting the duplicated entries in `track_id`

We create a Dataframe where containing only the rows where the  column has duplicates and check its dimensions to assert the number of duplicated rows.

In [112]:
duplicates_id = df[df.duplicated(subset=['track_id'], keep=False)]
print(f"The Dataframe with duplicates has {duplicates_id.shape[0]} rows.")

The Dataframe with duplicates has 40108 rows.


Now we compute the difference in the number of rows and columns between our original DataFrame (`df`) and the filtered DataFrame of duplicates (`duplicates_id`). 

In [113]:
difference = (df.shape[0] - duplicates_id.shape[0])

# Compute the percentage of non-duplicated rows
percentage_non_duplicated = (difference / df.shape[0]) * 100

# Print the result
print(f"The number of non-duplicated rows is {difference}, which is {percentage_non_duplicated:.2f}% of the original Spotify DataFrame.")

The number of non-duplicated rows is 73441, which is 64.68% of the original Spotify DataFrame.


We need to check if pairs with duplicated `track_id` along with their respective `track_name` have a correspondence. To ensure that all pairs of duplicated `track_id`s have the same `track_name`, the data is grouped by `track_id` and we check if each group has only one unique `track_name`.

1. **`groupby('track_id')`**: Groups the DataFrame by `track_id`.
2. **`nunique()`**: Counts the number of unique `track_name` values in each group.
3. **Check for inconsistencies**: Identifies `track_id`s where there is more than one unique `track_name`.

In [106]:
grouped = df.groupby('track_id')['track_name'].nunique()

# Check for track_ids with more than one unique track_name
inconsistent = grouped[grouped > 1]

if inconsistent.empty:
    print("All duplicated track_ids have the same track_name.")
else:
    print("Some duplicated track_ids have inconsistent track_names.")
    print(inconsistent)


All duplicated track_ids have the same track_name.


As all duplicated `track_id`s have the same `track_name`s, we need to further inspect if there is anything that differentiates this duplicate tracks. 

In [107]:
# Group by 'track_id' and check for identical rows within each group
identical_groups = duplicates_id.groupby('track_id').filter(
    lambda group: group.drop_duplicates().shape[0] == 1
)

if identical_groups.empty:
    print("No duplicates are fully identical across all fields.")
else:
    print("Fully identical rows:")
    print(identical_groups.shape[0])

No duplicates are fully identical across all fields.


In [109]:
def check_inconsistencies(df, id_col='track_id'):
    """Versión con mensajes de diagnóstico"""
    dup_mask = df.duplicated(subset=id_col, keep=False)
    print(f"Total registros duplicados: {dup_mask.sum()}")
    
    if not dup_mask.any():
        print("No hay duplicados para analizar")
        return pd.DataFrame(columns=[id_col, 'inconsistent_columns'])
    
    results = []
    for track_id in df.loc[dup_mask, id_col].unique():
        group = df[df[id_col] == track_id]
        inconsistent = [col for col in group.columns 
                       if col != id_col and group[col].nunique() > 1]
        if inconsistent:
            results.append({
                id_col: track_id,
                'inconsistent_columns': ', '.join(inconsistent),
                'n_duplicates': len(group),
                'example_values': str(group[inconsistent[0]].unique()[:3])  # Muestra primeros valores
            })
    
    if not results:
        print("Duplicados encontrados pero son completamente consistentes en todas las columnas")
    
    return pd.DataFrame(results)

In [110]:
inconsistencies = check_inconsistencies(duplicates_id)
if inconsistencies.empty:
    print("No se encontraron inconsistencias en duplicados")
else:
    print("Inconsistencias encontradas:")
    display(inconsistencies)

Total registros duplicados: 40108
Inconsistencias encontradas:


,track_id,inconsistent_columns,n_duplicates,example_values
0,5SuOikwiRyPMVoIQDJUgSV,track_genre,4,['acoustic' 'j-pop' 'singer-songwriter']
1,4qPNDBW1i3p13qLCt0Ki3A,track_genre,2,['acoustic' 'chill']
2,01MVOl9KtVTNfFiBU9I7dc,track_genre,2,['acoustic' 'indie-pop']
3,6Vc5wAMmXdKIAM7WUoEb7N,track_genre,2,['acoustic' 'piano']
4,1EzrEOXmMH3G43AXT1y7pA,track_genre,2,['acoustic' 'rock']
...,...,...,...,...
16294,79cxnmnGiC0qZfxi5ogp4j,track_genre,2,['techno' 'trance']
16295,1B0FEDRzzN5GP7HGZZfNQl,track_genre,2,['techno' 'trance']
16296,4D41idYLHmXYGaHZeRWtPT,track_genre,2,['techno' 'trip-hop']
16297,27nGU2v3syK7aU3AVY2vUO,track_genre,2,['techno' 'trance']


In [ ]:
def handle_duplicates_simple(df, id_col='track_id', strategy='first'):
    """
    Maneja duplicados de forma eficiente con estrategias básicas.
    
    Args:
        df: DataFrame de pandas
        id_col: Columna para identificar duplicados (default 'track_id')
        strategy: 'first' (keep first), 'last' (keep last), 
                 'mean' (promedio numérico/moda categórica),
                 'concat' (une valores únicos con '|')
    
    Returns:
        DataFrame procesado sin duplicados
    """
    # Verificación rápida de duplicados
    dup_mask = df.duplicated(subset=id_col, keep=False)
    
    if not dup_mask.any():
        return df.copy()
    
    # Estrategias simples
    if strategy in ['first', 'last']:
        return df.drop_duplicates(subset=id_col, keep=strategy)
    
    # Estrategias que requieren agrupación
    grouped = df.groupby(id_col)
    
    if strategy == 'mean':
        return grouped.agg(lambda x: x.mean() if np.issubdtype(x.dtype, np.number) 
                          else x.mode()[0] if not x.mode().empty else x.iloc[0]).reset_index()
    
    if strategy == 'concat':
        return grouped.agg(lambda x: '|'.join(map(str, x.unique()))).reset_index()
    
    raise ValueError(f"Estrategia no válida: {strategy}. Usar 'first', 'last', 'mean' o 'concat'")

### Checking categorical values in `track_genre`

In [111]:
np.sort(df.track_genre.unique())

array(['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient',
       'anime', 'black-metal', 'bluegrass', 'blues', 'brazil',
       'breakbeat', 'british', 'cantopop', 'chicago-house', 'children',
       'chill', 'classical', 'club', 'comedy', 'country', 'dance',
       'dancehall', 'death-metal', 'deep-house', 'detroit-techno',
       'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm',
       'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk',
       'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove',
       'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle',
       'heavy-metal', 'hip-hop', 'honky-tonk', 'house', 'idm', 'indian',
       'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol',
       'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino',
       'malay', 'mandopop', 'metal', 'metalcore', 'minimal-techno', 'mpb',
       'new-age', 'opera', 'pagode', 'party', 'piano', 'pop', 'pop-film',
       'pow